### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load
folder = 'Resources'
school_data_to_load = os.path.join(folder,'schools_complete.csv') #"Resources/schools_complete.csv"
student_data_to_load = os.path.join(folder,'students_complete.csv') #"Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Renaming the columns of the data frame
school_data_complete = school_data_complete.rename(columns={'student_name':'Student', 'gender':'Gender',
                                                           'grade':'Grade', 'school_name':'School','reading_score':'Reading Score',
                                                           'math_score':'Math Score', 'type':'School type', 
                                                            'size':'School size', 'budget':'School Budget'})
#checking if the dataset is complete
school_data_complete.count()


Student ID       39170
Student          39170
Gender           39170
Grade            39170
School           39170
Reading Score    39170
Math Score       39170
School ID        39170
School type      39170
School size      39170
School Budget    39170
dtype: int64

In [3]:
#Total Number of schools
school_count = len(school_data_complete['School ID'].unique())
#Total Number of Students
students_count = len(school_data_complete['Student'])
#Total Budget
total_budget = sum(school_data_complete.groupby('School ID')['School Budget'].mean())
#Average Math Score
avg_math_score = school_data_complete['Math Score'].sum()/len(school_data_complete['Math Score'])
#Average Reading Score
avg_reading_score = school_data_complete['Reading Score'].sum()/len(school_data_complete['Reading Score'])
#% of students passing Math
passing_math = school_data_complete.loc[school_data_complete['Math Score']>=70,:]
passing_math_percentage = len(passing_math['Student ID'])/students_count*100
#% of student passing reading
passing_read = school_data_complete.loc[school_data_complete['Reading Score']>=70,:]
passing_read_percentage = len(passing_read['Student ID'])/students_count*100
#% of students passing math AND reading
passing_math_reading = pd.merge(passing_math, passing_read, on='Student ID', how='inner', suffixes =('_math','_reading'))
math_reading_percentage = len(passing_math_reading['Student ID'])/students_count*100

district_summary = pd.DataFrame.from_dict({"Total Schools": [school_count],
                           "Total Students": students_count,
                            "Total Budget": '${:0,.2f}'.format(total_budget),
                           "Average Math Score": avg_math_score,
                           "Average Reading Score": avg_reading_score,
                           "% Passing Math": passing_math_percentage,
                           "% Passing Reading": passing_read_percentage,
                           "% Overall Passing": math_reading_percentage})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
column_name = 'School'
#retrieving the school names
school_names = school_data_complete[column_name].unique()

dic = dict()
#populating the dictionary with the values of choice
for name in school_names:
    lists = []
    school_type = school_data_complete.loc[school_data_complete[column_name]== name, :]['School type'].unique()
    total_students = school_data_complete.loc[school_data_complete[column_name]== name, :]['Student'].count()
    total_school_budget = school_data_complete.loc[school_data_complete[column_name]== name, :]['School Budget'].mean()
    student_budget = total_school_budget/total_students
    school_math_percent = len(passing_math.loc[passing_math[column_name] == name, :]['Math Score'])/total_students*100
    school_read_percent = len(passing_read.loc[passing_read[column_name]== name, :]['Reading Score'])/total_students*100
#    school_math_reading = len(passing_math_reading.loc[passing_math_reading['School_x']==name+'_x',:])/total_students*100
    lists = [str(school_type).strip("['']")]+[total_students]+[total_school_budget]+[student_budget]+[school_math_percent]+[school_read_percent]
    dic[name] = lists
school_math_reading = passing_math_reading.groupby('School_math')['Student ID'].count()/school_data_complete.groupby('School')['Student ID'].count()*100
_school_summary = pd.DataFrame.from_dict(dic,orient='index', columns=['School Type','Total Students', 'Total School Budget', 'Per Student Budget','% Passing Math','% Passing Reading'])
school_avg_math_score = school_data_complete.groupby(column_name)['Math Score'].mean()
school_avg_reading_score = school_data_complete.groupby(column_name)['Reading Score'].mean()
score_df = pd.concat([pd.DataFrame(school_math_reading),school_avg_math_score,school_avg_reading_score],axis =1)
_school_summary = _school_summary.join(score_df, how='left').sort_index() 
print(_school_summary.columns)
_school_summary = _school_summary.rename(columns={'Student ID':'% Overall Passing', 'Math Score':'Average Math Score', 'Reading Score':'Average Reading Score'})


Index(['School Type', 'Total Students', 'Total School Budget',
       'Per Student Budget', '% Passing Math', '% Passing Reading',
       'Student ID', 'Math Score', 'Reading Score'],
      dtype='object')


In [5]:
school_summary = _school_summary.copy()

school_summary['Total School Budget'] = school_summary['Total School Budget'].map("${:,.2f}".format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map("${:.2f}".format)

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Average Math Score,Average Reading Score
Bailey High School,District,4976,"$3,124,928.00",$628.00,66.680064,81.933280,54.642283,77.048432,81.033963
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,94.133477,97.039828,91.334769,83.061895,83.975780
Figueroa High School,District,2949,"$1,884,411.00",$639.00,65.988471,80.739234,53.204476,76.711767,81.158020
Ford High School,District,2739,"$1,763,916.00",$644.00,68.309602,79.299014,54.289887,77.102592,80.746258
Griffin High School,Charter,1468,"$917,500.00",$625.00,93.392371,97.138965,90.599455,83.351499,83.816757
Hernandez High School,District,4635,"$3,022,020.00",$652.00,66.752967,80.862999,53.527508,77.289752,80.934412
Holden High School,Charter,427,"$248,087.00",$581.00,92.505855,96.252927,89.227166,83.803279,83.814988
Huang High School,District,2917,"$1,910,635.00",$655.00,65.683922,81.316421,53.513884,76.629414,81.182722
Johnson High School,District,4761,"$3,094,650.00",$650.00,66.057551,81.222432,53.539172,77.072464,80.966394
Pena High School,Charter,962,"$585,858.00",$609.00,94.594595,95.945946,90.540541,83.839917,84.044699


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [6]:
top_schools = school_summary.sort_values(by=['% Overall Passing'], ascending = False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Average Math Score,Average Reading Score
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,94.133477,97.039828,91.334769,83.061895,83.975780
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,93.272171,97.308869,90.948012,83.418349,83.848930
Griffin High School,Charter,1468,"$917,500.00",$625.00,93.392371,97.138965,90.599455,83.351499,83.816757
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,93.867718,96.539641,90.582567,83.274201,83.989488
Pena High School,Charter,962,"$585,858.00",$609.00,94.594595,95.945946,90.540541,83.839917,84.044699


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
bottom_schools = school_summary.sort_values(by=['% Overall Passing'], ascending = True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Average Math Score,Average Reading Score
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,66.366592,80.220055,52.988247,76.842711,80.744686
Figueroa High School,District,2949,"$1,884,411.00",$639.00,65.988471,80.739234,53.204476,76.711767,81.158020
Huang High School,District,2917,"$1,910,635.00",$655.00,65.683922,81.316421,53.513884,76.629414,81.182722
Hernandez High School,District,4635,"$3,022,020.00",$652.00,66.752967,80.862999,53.527508,77.289752,80.934412
Johnson High School,District,4761,"$3,094,650.00",$650.00,66.057551,81.222432,53.539172,77.072464,80.966394


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
# https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
#unutbu is the creator of these two functions. They were taken from the link above.
import re

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return ([atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text)]) 

In [9]:
# finding the unique grades
grade = list(school_data_complete['Grade'].unique())
#creating a multiple index series containing the average values for the math scores as a function of the school and the grade
math_score = school_data_complete.groupby(['School','Grade'])['Math Score'].mean()
# transforming the multiple index series in a multiple index dataframe
math_score = pd.DataFrame(math_score)

#separating the average values per grade and reconcatenating the extrapolated sieries in another dataframe
mgrades = []
for x in grade:
    mgrades += [math_score.xs(x, level = 1)]
mgrades_df = pd.concat(mgrades, axis=1)

# formatting the dataframe to look clean and nice
for x in range(len(grade)):
    mgrades_df.rename(columns={mgrades_df.columns[x]: grade[x] })
mgrades_df.columns=grade
mgrades_df = mgrades_df[sorted(grade,key = natural_keys)]
mgrades_df.index.name = None
#showing the final dataframe
mgrades_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:
#creating a multiple index series containing the average values for the reading scores as a function of the school and the grade
reading_score = school_data_complete.groupby(['School','Grade'])['Reading Score'].mean()
reading_score = pd.DataFrame(reading_score)

#separating the average values per grade and reconcatenating the extrapolated sieries in another dataframe
grades = []
for x in grade:
    grades += [reading_score.xs(x, level = 1)]
grades_df = pd.concat(grades, axis=1)

# formatting the dataframe to look clean and nice
for x in range(len(grade)):
    grades_df.rename(columns={ grades_df.columns[x]: grade[x] })
grades_df.columns=grade
grades_df = grades_df[sorted(grade,key = natural_keys)]
grades_df.index.name = None

#showing the final dataframe
grades_df

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
def summary_binning(df, column_tobin, new_column_name, bins, bin_labels, ):
    #binning the array
    df[new_column_name] = pd.cut(df[column_tobin], bins, labels = bin_labels)
    #grouping by the variable of choide and calculating the average of the parameters
    new_df = df.groupby(new_column_name)[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading','% Overall Passing']].mean()
    #formatting the columns in the newly created dataframe
    for column in new_df.columns:
        new_df[column] = new_df[column].map('{:.2f}'.format)
    return new_df

In [12]:
# creating the bins
spending_bins = [0, 585, 630, 645, 680]
# creating the labels
spending_labels = ['<$585', '$585-630','$630-645','$645-680']
# running the function summary_binning to bin and format the summary result
school_spending_score = summary_binning(_school_summary, 'Per Student Budget', 'Spending Ranges per Student', spending_bins, spending_labels)
#printing the output on screen
school_spending_score

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges per Student,,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [13]:
# creating the bins
size_bins = [0, 1000, 2000, 5000]
# creating the labels
size_labels = ['Small (<1000)', 'Medium (1000-2000)','Large (2000-5000)']
# binning the dataframe named school_summary
school_size = summary_binning(_school_summary, 'Total Students', 'School Size', size_bins, size_labels)
#printing the output on screen
school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [14]:
#In this case the column with the binning value already exists.
#grouping by the school type and averaging is enough
school_type = _school_summary.groupby('School Type')[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading','% Overall Passing']].mean()
school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


## Trends observable from the analysis above

1. __*Charter Schools perform better than District Schools*__. Indeed, the top five schools are all charter schools while the bottom five are district schools. Additionally, the overall passing % for district schools is 53.67% while for the charter is 90.43%.
2. Surprisingly, **_the overall student performances decreased with increasing percentage of spending per student_**.
3. There is no difference in performance between small and medium schools. However, **large schools** (with more than 2000 students) **showed worse performances than small and medium schools**. 